In [22]:
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

pretrained = 'mdhugol/indonesia-bert-sentiment-classification'
model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)

sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

c:\Users\muhfa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\muhfa\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
import tqdm

df_comment = pd.read_csv('data/bank_sentiment_test.csv', encoding='utf-8')
df_comment = df_comment[['teks']]

label_index = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}
status_list = []
score_list = []

for comment in tqdm.tqdm(df_comment['teks']):
    if comment == "" :
        status_list.append("")
        score_list.append("")
    else : 
        try:
            result = sentiment_analysis(str(comment))
            status_list.append(label_index[result[0]['label']])
            score_list.append(result[0]['score'])
        except:
            status_list.append("")
            score_list.append("")

    
df_output = pd.DataFrame({'comment' : list(df_comment['teks']), 'sentimen' : status_list, 'score' : score_list})

100%|██████████| 300/300 [00:25<00:00, 11.73it/s]


In [25]:
df_output['label'] = pd.read_csv('data/bank_sentiment_test.csv', encoding='utf-8')['sentimen']
df_output['context'] = pd.read_csv('data/bank_sentiment_test.csv', encoding='utf-8')['target_bank']
df_output.to_csv('bank_sentiment_test_with_sentiment.csv', index=False, encoding='utf-8')

# Evaluataion

In [30]:
import pandas as pd
df = pd.read_csv('bank_sentiment_test_with_sentiment.csv', encoding='utf-8')
mapping = {'positive': "positif", 'neutral': "netral", 'negative': "negatif"}
df['sentimen'] = df['sentimen'].map(mapping).fillna(df['sentimen'])

In [32]:
accuracy = (df['sentimen'] == df['label']).mean()
print(f"Accuracy of sentiment analysis: {accuracy:.2%}")

Accuracy of sentiment analysis: 52.67%


In [33]:
df_non_campuran = df[df['label']!='campuran']
accuracy_non_campuran = (df_non_campuran['sentimen'] == df_non_campuran['label']).mean()
print(f"Accuracy of sentiment analysis (non-campuran): {accuracy_non_campuran:.2%}")


Accuracy of sentiment analysis (non-campuran): 65.56%


In [34]:
df_non_campuran[df_non_campuran['label']!=df_non_campuran['sentimen']].head()

,comment,sentimen,score,label,context
2,"Baru seminggu di Bank BCA, masih adaptasi tapi...",netral,0.509538,positif,Bank BCA
4,"Honestly, saat transfer ke orang tua, night mo...",negatif,0.838103,positif,Bank BCA
6,"Kalau menurut saya, ketika lagi traveling, rua...",negatif,0.994011,positif,Bank BCA
8,"Akhir-akhir ini, waktu mau bayar tagihan, biay...",positif,0.933777,negatif,Bank BRI
9,"Terus terang, saat transfer ke orang tua, laya...",negatif,0.985931,positif,Bank BNI


In [35]:
for i, row in df_non_campuran[df_non_campuran['label']!=df_non_campuran['sentimen']].iterrows():
    print(f"Comment: {row['comment']}\nLabel: {row['label']}\nSentiment: {row['sentimen']}\nContext: {row['context']}\n")

Comment: Baru seminggu di Bank BCA, masih adaptasi tapi promising
Label: positif
Sentiment: netral
Context: Bank BCA

Comment: Honestly, saat transfer ke orang tua, night mode BNI Mobile Banking malah bikin mata pusing, kontras kurang parah. Mending pindah ke Bank BCA aja kayaknya.
Label: positif
Sentiment: negatif
Context: Bank BCA

Comment: Kalau menurut saya, ketika lagi traveling, ruang ATM Bank BNI sering kotor dan kurang terawat total. Mending pindah ke Bank BCA aja kayaknya.
Label: positif
Sentiment: negatif
Context: Bank BCA

Comment: Akhir-akhir ini, waktu mau bayar tagihan, biaya admin Bank BNI cukup terjangkau, sesuai dengan layanannya. Bank BNI masih di bawah Bank BRI sih.
Label: negatif
Sentiment: positif
Context: Bank BRI

Comment: Terus terang, saat transfer ke orang tua, layar sentuh di ATM Bank Mandiri sering tidak responsif sangat. Mending pindah ke Bank BNI aja kayaknya.
Label: positif
Sentiment: negatif
Context: Bank BNI

Comment: Pengalaman pribadiku sih, ATM Bank 